In [2]:
import import_ipynb
from news_classes import *
import math
from textblob import TextBlob as tb
import os
import pickle
from collections import Counter
from textblob import TextBlob as tb

importing Jupyter notebook from news_classes.ipynb


In [3]:
idf_path='idf_30_9_2018.pkl'
with open(idf_path, 'rb') as input:
    idfblob = pickle.load(input)

In [4]:
def idf(word):
    return math.log(idfblob.size() / (1 + idfblob.getIdf(word)))

In [144]:
def sent_cosine(sentence1, sentence2):
    count_sent1=Counter(sentence1.words)
    count_sent2=Counter(sentence2.words)
    
    up=0
    downright=0
    downleft=0
    
    words=set()
    for word in count_sent1.keys():
        words.add(word)
    for word in count_sent2.keys():
        words.add(word)
    
    for word in words:
        cnt1=count_sent1.get(word)
        cnt2=count_sent2.get(word)
        if cnt1 is None:
            cnt1=0            
        elif cnt2 is None:
            cnt2=0
                
        up+=cnt1*cnt2*idf(word)*idf(word)
        downright+=cnt1*idf(word)*cnt1*idf(word)
        downleft+=cnt2*idf(word)*cnt2*idf(word)
        
    try:
        return up/(math.sqrt(downleft)*math.sqrt(downright))
    except:
        return 0

In [158]:
blob=tb("The BNP chief challenged the legality of a trial court decision to continue the proceedings in her absence, and prayed to issue a rule on Anti-Corruption Commission (ACC) and the government to explain why the lower court's decision should not be scrapped. Trial cannot continue if the accused is sick, her lawyer Barrister Kaiser Kamal said, adding, that setting up a court in old Dhaka jail is unlawful and unconstitutional. Meanwhile, a Dhaka court yesterday fixed October 24 for framing charges against Khaleda in cases filed for undermining the national flag and observing “fake birthday” on August 15. ")

In [159]:
sent_cosine(blob.sentences[0], blob.sentences[1])

0.0 0.0 2.370509804558544
0.0 0.0 66.03489289066168
0.0 0.0 66.13068383384787
0.0 1.1604873068475838 66.13068383384787
0.0 28.943360831269857 66.13068383384787
0.0 43.536626360561456 66.13068383384787
0.0 44.73906778141544 66.13068383384787
0.0 57.9536810171101 66.13068383384787
0.0 58.880028248441555 66.13068383384787
0.0 58.880028248441555 72.47811489875586
0.17014577989693308 58.92256469341579 73.1586980183436
0.17014577989693308 58.92256469341579 73.21284531115501
0.17014577989693308 122.58694777951892 73.21284531115501
0.17014577989693308 122.58694777951892 136.87722839725814
0.17014577989693308 122.58694777951892 141.7781621621894
0.17014577989693308 122.58694777951892 178.17629152184793
0.17014577989693308 146.0464609042817 178.17629152184793
0.17014577989693308 172.52554386316305 178.17629152184793
0.17014577989693308 172.92785157764885 178.17629152184793
0.17014577989693308 172.92785157764885 241.84067460795106
0.17014577989693308 172.92785157764885 242.216668653884
0.17014577

0.04726398312965018